In [ ]:
# import pyspark class Row from module sql
from pyspark.sql import *
spark = SparkSession.builder.getOrCreate()
# Create Example Data - Departments and Employees

# Create the Departments
department1 = Row(id='101', name='Computer Science')
department2 = Row(id='102', name='Mechanical Engineering')
department3 = Row(id='103', name='Theater and Drama')
department4 = Row(id='104', name='Indoor Recreation')

# Create the Employees
Employee = Row("firstName", "lastName", "email", "salary","deptId")
employee1 = Employee('michael', 'armbrust', 'no-reply@berkeley.edu', 100000,101)
employee2 = Employee('xiangrui', 'meng', 'no-reply@stanford.edu', 120000,101)
employee3 = Employee('matei', None, 'no-reply@waterloo.edu', 140000,102)
employee4 = Employee(None, 'wendell', 'no-reply@berkeley.edu', 160000,102)
employee5 = Employee('michael', 'jackson', 'no-reply@neverla.nd', 80000,103)
employee6 = Employee('john', 'doe', 'no-reply@unclesame.usa.gov', 800000,101)

depts = [department1, department1,department3,department4]
employees = [employee1, employee2,employee3,employee4,employee5,employee6]

deptsDF = spark.createDataFrame(depts)
empsDF = spark.createDataFrame(employees)

deptsDF.show()
empsDF.show()

In [ ]:
import re 
from shutil import copyfile
from pyspark.dbutils import DBUtils

dbutils = DBUtils(spark.sparkContext)
# print(dbutils.fs.ls("dbfs:/"))

global fileprefix
global deptFilePath
deptFilePath = "dept-sample.json"

fileprefix = "dbfs:/sample-data"
print('deptsDF count is : ', deptsDF.count())

deptsDF.coalesce(1).write.mode("overwrite").option("header", "true").json(fileprefix+".tmp")
partition_path = dbutils.fs.ls(fileprefix+".tmp/")
path = [file for file in partition_path if file.name.endswith(".json")][0].path
# path = re.sub('dbfs:', '/dbfs', path)
deptFilePath = "dbfs:/" + deptFilePath
# print('Path is : ', path)
# print('deptFilePath is : ', deptFilePath)


try:
    dbutils.fs.cp(path, deptFilePath)
    dbutils.fs.rm(path)
    print(dbutils.fs.head(deptFilePath))

except:
    print("Unexpected error:", sys.exc_info())
    exit(1)



In [ ]:
global empFilePath
empFilePath = "employee-sample.json"

print('empsDF count is : ', empsDF.count())

empsDF.coalesce(1).write.mode("overwrite").option("header", "true").json(fileprefix+".tmp")
partition_path = dbutils.fs.ls(fileprefix+".tmp/")
path = [file for file in partition_path if file.name.endswith(".json")][0].path
empFilePath = "dbfs:/" + empFilePath
# print('Path is : ', path)
# print('deptFilePath is : ', empFilePath)

try:
    dbutils.fs.cp(path, empFilePath)
    dbutils.fs.rm(path)
    print(dbutils.fs.head(empFilePath))
except:
    print("Unexpected error:", sys.exc_info())
    exit(1)



In [ ]:
# dbutils.fs.rm("dept-sample.json.json")
# deptFilePath = re.sub('/dbfs', '', deptFilePath)
df = spark.read.option("inferSchema",True).json(deptFilePath)
print('df count is : ', df.count())
df.show()

In [ ]:
# empFilePath = re.sub('/dbfs', '', empFilePath)
print(empFilePath)
empDf1 = spark.read.option("inferSchema",True).json(empFilePath)
print('empDf1 count is : ', empDf1.count())
empDf1.show()